In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip', parse_dates=['Dates'])
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip', parse_dates=['Dates'], index_col = 'Id')
samSub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
a = list(test)
train_data = train[a]
main = pd.concat([train_data, test])
main.head(3)
df = main.copy()
df.head()

# > **Lets seperate the target Columnns**

In [ ]:
tar = train['Category']
tar.head()

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder
LB = LabelEncoder()
target = LB.fit_transform(tar)
print(LB.classes_)
temp = pd.DataFrame(tar, target)
temp.head(3)

In [ ]:
date = pd.to_datetime(main['Dates'])
df['Date'] = date.dt.date
df['Year'] = date.dt.year
df['Month'] = date.dt.month
df['Day'] = date.dt.day
df['Hour'] = date.dt.hour
df.drop('Dates', axis = 1, inplace = True)

In [ ]:
df.head(3)

In [ ]:
date = df.groupby('Date').count().iloc[:, 0]

In [ ]:
sns.kdeplot(data = date, shade = True)
plt.axvline(x = date.median(), ymax = 0.95, linestyle = '--')
plt.annotate('Median' + str(date.median()), xy = (date.median(), 0.005))

The number of crimes per day has a normal distribution, with a median of 389.

In [ ]:
lb = LabelEncoder()
df['PdDis'] = lb.fit_transform(df['PdDistrict'])
sns.countplot(df['PdDis'])

In [ ]:
df[['PdDistrict', 'PdDis']].head(10)

In [ ]:
df.drop('PdDistrict', axis = 1, inplace = True)

In [ ]:
df.head(3)

In [ ]:
df['DayWeek'] = lb.fit_transform(df['DayOfWeek'])
df[['DayOfWeek', 'DayWeek']].head(10)

In [ ]:
df.drop('DayOfWeek', axis = 1, inplace = True)

In [ ]:
df.head(3)

In [ ]:
df['Block'] = df['Address'].str.contains('block', case = False)
df['ST'] = df['Address'].str.contains('ST', case = False)
df.drop('Address', axis = 1, inplace = True)

In [ ]:
df['Block'] = lb.fit_transform(df['Block'])
df['ST'] = lb.fit_transform(df['ST'])

In [ ]:
df.head()

In [ ]:
print(df['X'].min(), df['X'].max())
print(df['Y'].min(), df['Y'].max())

In [ ]:
medX = df[df['X'] < -120.5]['X'].median()
medY = df[df['Y'] < 90]['Y'].median()
df.loc[df['X'] >= -120.5, 'X'] = medX
df.loc[df['Y'] >= 90, 'Y'] = medY

In [ ]:
df['X+Y'] = df['X'] + df['Y']
df['X-Y'] = df['X'] - df['Y']

In [ ]:
df.drop('Date', axis = 1, inplace = True)

In [ ]:
data_train = df[:train.shape[0]]
data_test = df[train.shape[0]:]
data_train.head()

In [ ]:
import lightgbm as lg
trains = lg.Dataset(data_train, label = target, categorical_feature=['PdDis', 'DayWeek'])
params = {
    'boosting':'gbdt',
    'objective':'multiclass',
    'num_class':39,
    'max_delta_step':0.9,
    'min_data_in_leaf': 20,
    'learning_rate': 0.4,
    'max_bin': 480,
    'num_leaves': 45,
    'verbose' : 1
}
bst = lg.train(params, trains, 120)

In [ ]:
prediction = bst.predict(data_test)

In [ ]:
result1 = pd.DataFrame(prediction, columns = LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=data_test.index) 
result1.head()
result1.to_csv('submission-1', index_label = 'Id')